In [1]:
import os
import sys
import numpy as np
import gc
import h5py
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from models.networks_2022 import BranchedNetwork
from data.ReconstructionTrainingDataset import CleanSoundsDataset
from data.NoisyDataset import NoisyDataset, FullNoisyDataset, LargeNoisyDataset
from data.MergedNoisyDataset import MergedNoisyDataset

# Global configurations

In [7]:
# Main args
SAME_PARAM = False           # to use the same parameters for all pcoders or not
noise_types = ['AudScene']

In [8]:
# Dataset configuration
snr_levels = [-9.]
BATCH_SIZE = 10
NUM_WORKERS = 2

In [9]:
# Other training params
EPOCH = 15
FF_START = True             # to start from feedforward initialization
MAX_TIMESTEP = 5

In [10]:
# Path names
engram_dir = '/mnt/smb/locker/abbott-locker/hcnn/'
checkpoints_dir = f'{engram_dir}checkpoints/'
tensorboard_dir = f'{engram_dir}tensorboard/fb_ablation/'

# Load network arguments

In [12]:
if SAME_PARAM:
    from pbranchednetwork_shared import PBranchedNetwork_SharedSameHP
    PNetClass = PBranchedNetwork_SharedSameHP
    pnet_name = 'pnet'
    fb_state_dict_path = f'{checkpoints_dir}{pnet_name}/{pnet_name}-shared-50-regular.pth'
else:
    from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
    PNetClass = PBranchedNetwork_AllSeparateHP
    pnet_name = 'pnet'
    fb_state_dict_path = f'{checkpoints_dir}{pnet_name}/{pnet_name}-50-regular.pth'
fb_state_dict = torch.load(fb_state_dict_path)

# Helper functions

In [13]:
def load_pnet(
        net, state_dict, build_graph, random_init,
        ff_multiplier, fb_multiplier, er_multiplier,
        same_param, device='cuda:0'):
    
    pnet = PNetClass(
        net, build_graph=build_graph, random_init=random_init,
        ff_multiplier=ff_multiplier, fb_multiplier=fb_multiplier, er_multiplier=er_multiplier
        )

    pnet.load_state_dict(state_dict)
    pnet.eval()
    pnet.to(device)
    return pnet

In [14]:
def evaluate(net, epoch, dataloader, timesteps, loss_function, writer=None, tag='Clean'):
    test_loss = np.zeros((timesteps+1,))
    correct   = np.zeros((timesteps+1,))
    for (images, labels) in dataloader:
        images = images.cuda()
        labels = labels.cuda()
        
        with torch.no_grad():
            for tt in range(timesteps+1):
                if tt == 0:
                    outputs, _ = net(images)
                else:
                    outputs, _ = net()
                
                loss = loss_function(outputs, labels)
                test_loss[tt] += loss.item()
                _, preds = outputs.max(1)
                correct[tt] += preds.eq(labels).sum()

    print()
    for tt in range(timesteps+1):
        test_loss[tt] /= len(dataloader.dataset)
        correct[tt] /= len(dataloader.dataset)
        print('Test set t = {:02d}: Average loss: {:.4f}, Accuracy: {:.4f}'.format(
            tt,
            test_loss[tt],
            correct[tt]
        ))
        if writer is not None:
            writer.add_scalar(
                f"{tag}Perf/Epoch#{epoch}",
                correct[tt], tt
                )
    print()


In [15]:
def train(net, epoch, dataloader, timesteps, loss_function, optimizer, writer=None):
    for batch_index, (images, labels) in enumerate(dataloader):
        net.reset()

        labels = labels.cuda()
        images = images.cuda()

        ttloss = np.zeros((timesteps+1))
        optimizer.zero_grad()

        for tt in range(timesteps+1):
            if tt == 0:
                outputs, _ = net(images)
                loss = loss_function(outputs, labels)
                ttloss[tt] = loss.item()
            else:
                outputs, _ = net()
                current_loss = loss_function(outputs, labels)
                ttloss[tt] = current_loss.item()
                loss += current_loss
        
        loss.backward()
        optimizer.step()
        net.update_hyperparameters()
            
        print(f"Training Epoch: {epoch} [{batch_index * BATCH_SIZE + len(images)}/{len(dataloader.dataset)}]\tLoss: {loss.item():0.4f}\tLR: {optimizer.param_groups[0]['lr']:0.6f}")
        for tt in range(timesteps+1):
            print(f'{ttloss[tt]:0.4f}\t', end='')
        print()
        if writer is not None:
            writer.add_scalar(
                f"TrainingLoss/CE", loss.item(),
                (epoch-1)*len(dataloader) + batch_index
                )


In [16]:
def log_hyper_parameters(net, epoch, sumwriter, same_param=True):
    if same_param:
        sumwriter.add_scalar(f"HyperparamRaw/feedforward", getattr(net,f'ff_part').item(), epoch)
        sumwriter.add_scalar(f"HyperparamRaw/feedback",    getattr(net,f'fb_part').item(), epoch)
        sumwriter.add_scalar(f"HyperparamRaw/error",       getattr(net,f'errorm').item(), epoch)
        sumwriter.add_scalar(f"HyperparamRaw/memory",      getattr(net,f'mem_part').item(), epoch)

        sumwriter.add_scalar(f"Hyperparam/feedforward", getattr(net,f'ffm').item(), epoch)
        sumwriter.add_scalar(f"Hyperparam/feedback",    getattr(net,f'fbm').item(), epoch)
        sumwriter.add_scalar(f"Hyperparam/error",       getattr(net,f'erm').item(), epoch)
        sumwriter.add_scalar(f"Hyperparam/memory",      1-getattr(net,f'ffm').item()-getattr(net,f'fbm').item(), epoch)
    else:
        for i in range(1, net.number_of_pcoders+1):
            sumwriter.add_scalar(f"Hyperparam/pcoder{i}_feedforward", getattr(net,f'ffm{i}').item(), epoch)
            if i < net.number_of_pcoders:
                sumwriter.add_scalar(f"Hyperparam/pcoder{i}_feedback", getattr(net,f'fbm{i}').item(), epoch)
            else:
                sumwriter.add_scalar(f"Hyperparam/pcoder{i}_feedback", 0, epoch)
            sumwriter.add_scalar(f"Hyperparam/pcoder{i}_error", getattr(net,f'erm{i}').item(), epoch)
            if i < net.number_of_pcoders:
                sumwriter.add_scalar(f"Hyperparam/pcoder{i}_memory",      1-getattr(net,f'ffm{i}').item()-getattr(net,f'fbm{i}').item(), epoch)
            else:
                sumwriter.add_scalar(f"Hyperparam/pcoder{i}_memory",      1-getattr(net,f'ffm{i}').item(), epoch)


# Main hyperparameter optimization script

In [17]:
def train_and_eval(noise_type, snr_level):
    if noise_type == 'Merged':
        noisy_ds = MergedNoisyDataset(subset=0.9, train=True)
        noise_loader = torch.utils.data.DataLoader(
            noisy_ds,  batch_size=BATCH_SIZE,
            shuffle=True, drop_last=False,
            num_workers=NUM_WORKERS
            )
        eval_ds = MergedNoisyDataset(subset=0.9, train=False)
        eval_loader = torch.utils.data.DataLoader(
            eval_ds,  batch_size=BATCH_SIZE,
            shuffle=True, drop_last=False,
            num_workers=NUM_WORKERS
            )
    else:
        noisy_ds = NoisyDataset(bg=noise_type, snr=snr_level)
        noise_loader = torch.utils.data.DataLoader(
            noisy_ds,  batch_size=BATCH_SIZE,
            shuffle=True, drop_last=False,
            num_workers=NUM_WORKERS
            )
        eval_loader = noise_loader

    # Set up logs and network for training
    net_dir = f'hyper_{noise_type}_snr{snr_level}'
    if FF_START:
        net_dir += '_FFstart'
    if SAME_PARAM:
        net_dir += '_shared'

    sumwriter = SummaryWriter(f'{tensorboard_dir}{net_dir}')
    net = BranchedNetwork() # Load original network
    net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))
    pnet_fw = load_pnet( # Load FF PNet
        net, fb_state_dict, build_graph=False, random_init=(not FF_START),
        ff_multiplier=1.0, fb_multiplier=0.0, er_multiplier=0.0,
        same_param=SAME_PARAM, device='cuda:0'
        )
    loss_function = torch.nn.CrossEntropyLoss()
    evaluate(
        pnet_fw, 0, eval_loader, 1,
        loss_function,
        writer=sumwriter, tag='FeedForward')
    del pnet_fw
    gc.collect()

    # Load PNet for hyperparameter optimization
    net = BranchedNetwork()
    net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))
    pnet = load_pnet(
        net, fb_state_dict, build_graph=True, random_init=(not FF_START),
        ff_multiplier=0.33, fb_multiplier=0.33, er_multiplier=0.0,
        same_param=SAME_PARAM, device='cuda:0'
        )

    # Set up loss function and hyperparameters
    loss_function = torch.nn.CrossEntropyLoss()
    hyperparams = [*pnet.get_hyperparameters()]
    import pdb; pdb.set_trace()
    if SAME_PARAM:
        optimizer = torch.optim.Adam([
            {'params': hyperparams[:-1], 'lr':0.01},
            {'params': hyperparams[-1:], 'lr':0.0001}], weight_decay=0.00001)
    else:
        fffbmem_hp = []
        erm_hp = []
        for pc in range(pnet.number_of_pcoders):
            fffbmem_hp.extend(hyperparams[pc*4:pc*4+3])
            erm_hp.append(hyperparams[pc*4+3])
        optimizer = torch.optim.Adam([
            {'params': fffbmem_hp, 'lr':0.01},
            {'params': erm_hp, 'lr':0.0001}], weight_decay=0.00001)

    # Log initial hyperparameter and eval values
    log_hyper_parameters(pnet, 0, sumwriter, same_param=SAME_PARAM)
    hps = pnet.get_hyperparameters_values()
    print(hps)
    evaluate(
        pnet, 0, eval_loader,
        MAX_TIMESTEP, loss_function,
        writer=sumwriter, tag='Noisy'
        )

    # Run epochs
    for epoch in range(1, EPOCH+1):
        train(
            pnet, epoch, noise_loader,
            MAX_TIMESTEP, loss_function, optimizer,
            writer=sumwriter
            )
        log_hyper_parameters(pnet, epoch, sumwriter, same_param=SAME_PARAM)
        hps = pnet.get_hyperparameters_values()
        print(hps)

        evaluate(
            pnet, epoch, eval_loader,
            MAX_TIMESTEP, loss_function,
            writer=sumwriter, tag='Noisy'
            )
    sumwriter.close()

In [ ]:
for noise_type in noise_types:
    for snr_level in snr_levels:
        print("=====================")
        print(f'{noise_type}, for SNR {snr_level}')
        print("=====================")
        train_and_eval(noise_type, snr_level)

AudScene, for SNR -9.0


/share/ctn/users/cf2794/Code/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/ctn/users/cf2794/Code/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')
/home/cf2794/.conda/envs/hcnn/lib/python3.7/site-packages/predify/modules/base.py:260: UserWarning: Using a target size (torch.Size([10, 164, 400])) that is different to the input size (torch.Size([10, 1, 164, 400])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  self.prediction_error  = nn.functional.mse_loss(self.prd, target)
/home/cf2794/.conda/envs/hcnn/lib/python3.7/site-packages/predify/modules/base.py:260: UserWarning: Using a target size (torch.Size([3, 164, 400])) that is different to the input size (torch.Size([3, 1, 164, 400])). This will likely


Test set t = 00: Average loss: 0.5165, Accuracy: 0.1812
Test set t = 01: Average loss: 0.5002, Accuracy: 0.1865

> /tmp/ipykernel_36653/3147566134.py(60)train_and_eval()
     58     hyperparams = [*pnet.get_hyperparameters()]
     59     import pdb; pdb.set_trace()
---> 60     if SAME_PARAM:
     61         optimizer = torch.optim.Adam([
     62             {'params': hyperparams[:-1], 'lr':0.01},

ipdb> print(hyperparams)
[Parameter containing:
tensor(-0.8473, device='cuda:0', requires_grad=True), Parameter containing:
tensor(-0.8473, device='cuda:0', requires_grad=True), Parameter containing:
tensor(-0.4055, device='cuda:0', requires_grad=True), Parameter containing:
tensor(0.0100, device='cuda:0', requires_grad=True), Parameter containing:
tensor(-0.8473, device='cuda:0', requires_grad=True), Parameter containing:
tensor(-0.8473, device='cuda:0', requires_grad=True), Parameter containing:
tensor(-0.4055, device='cuda:0', requires_grad=True), Parameter containing:
tensor(0.0100, dev